In [16]:
from huggingface_hub import login

login()

In [17]:
from datasets import load_dataset
from transformers import AutoTokenizer, M2M100ForConditionalGeneration, AutoModelForCausalLM, BitsAndBytesConfig, AutoModelForSeq2SeqLM, NllbTokenizerFast
from tokenization_small100 import SMALL100Tokenizer
from peft import LoraModel

In [18]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype="bfloat16",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)

In [19]:
from peft import LoraConfig, get_peft_model

FAIRSEQ_LANGUAGE_CODES = ['ace_Arab', 'ace_Latn', 'acm_Arab', 'acq_Arab', 'aeb_Arab', 'afr_Latn', 'ajp_Arab', 'aka_Latn', 'amh_Ethi', 'apc_Arab', 'arb_Arab', 'ars_Arab', 'ary_Arab', 'arz_Arab', 'asm_Beng', 'ast_Latn', 'awa_Deva', 'ayr_Latn', 'azb_Arab', 'azj_Latn', 'bak_Cyrl', 'bam_Latn', 'ban_Latn', 'bel_Cyrl', 'bem_Latn', 'ben_Beng', 'bho_Deva', 'bjn_Arab', 'bjn_Latn', 'bod_Tibt', 'bos_Latn', 'bug_Latn', 'bul_Cyrl', 'cat_Latn', 'ceb_Latn', 'ces_Latn', 'cjk_Latn', 'ckb_Arab', 'crh_Latn', 'cym_Latn', 'dan_Latn', 'deu_Latn', 'dik_Latn', 'dyu_Latn', 'dzo_Tibt', 'ell_Grek', 'eng_Latn', 'epo_Latn', 'est_Latn', 'eus_Latn', 'ewe_Latn', 'fao_Latn', 'pes_Arab', 'fij_Latn', 'fin_Latn', 'fon_Latn', 'fra_Latn', 'fur_Latn', 'fuv_Latn', 'gla_Latn', 'gle_Latn', 'glg_Latn', 'grn_Latn', 'guj_Gujr', 'hat_Latn', 'hau_Latn', 'heb_Hebr', 'hin_Deva', 'hne_Deva', 'hrv_Latn', 'hun_Latn', 'hye_Armn', 'ibo_Latn', 'ilo_Latn', 'ind_Latn', 'isl_Latn', 'ita_Latn', 'jav_Latn', 'jpn_Jpan', 'kab_Latn', 'kac_Latn', 'kam_Latn', 'kan_Knda', 'kas_Arab', 'kas_Deva', 'kat_Geor', 'knc_Arab', 'knc_Latn', 'kaz_Cyrl', 'kbp_Latn', 'kea_Latn', 'khm_Khmr', 'kik_Latn', 'kin_Latn', 'kir_Cyrl', 'kmb_Latn', 'kon_Latn', 'kor_Hang', 'kmr_Latn', 'lao_Laoo', 'lvs_Latn', 'lij_Latn', 'lim_Latn', 'lin_Latn', 'lit_Latn', 'lmo_Latn', 'ltg_Latn', 'ltz_Latn', 'lua_Latn', 'lug_Latn', 'luo_Latn', 'lus_Latn', 'mag_Deva', 'mai_Deva', 'mal_Mlym', 'mar_Deva', 'min_Latn', 'mkd_Cyrl', 'plt_Latn', 'mlt_Latn', 'mni_Beng', 'khk_Cyrl', 'mos_Latn', 'mri_Latn', 'zsm_Latn', 'mya_Mymr', 'nld_Latn', 'nno_Latn', 'nob_Latn', 'npi_Deva', 'nso_Latn', 'nus_Latn', 'nya_Latn', 'oci_Latn', 'gaz_Latn', 'ory_Orya', 'pag_Latn', 'pan_Guru', 'pap_Latn', 'pol_Latn', 'por_Latn', 'prs_Arab', 'pbt_Arab', 'quy_Latn', 'ron_Latn', 'run_Latn', 'rus_Cyrl', 'sag_Latn', 'san_Deva', 'sat_Beng', 'scn_Latn', 'shn_Mymr', 'sin_Sinh', 'slk_Latn', 'slv_Latn', 'smo_Latn', 'sna_Latn', 'snd_Arab', 'som_Latn', 'sot_Latn', 'spa_Latn', 'als_Latn', 'srd_Latn', 'srp_Cyrl', 'ssw_Latn', 'sun_Latn', 'swe_Latn', 'swh_Latn', 'szl_Latn', 'tam_Taml', 'tat_Cyrl', 'tel_Telu', 'tgk_Cyrl', 'tgl_Latn', 'tha_Thai', 'tir_Ethi', 'taq_Latn', 'taq_Tfng', 'tpi_Latn', 'tsn_Latn', 'tso_Latn', 'tuk_Latn', 'tum_Latn', 'tur_Latn', 'twi_Latn', 'tzm_Tfng', 'uig_Arab', 'ukr_Cyrl', 'umb_Latn', 'urd_Arab', 'uzn_Latn', 'vec_Latn', 'vie_Latn', 'war_Latn', 'wol_Latn', 'xho_Latn', 'ydd_Hebr', 'yor_Latn', 'yue_Hant', 'zho_Hans', 'zho_Hant', 'zul_Latn']  # fmt: skip

tokenizer = NllbTokenizerFast.from_pretrained("facebook/nllb-200-distilled-600M",
                                                src_lang="eng_Latn",
                                                tgt_lang="bicol",)
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/nllb-200-distilled-600M", quantization_config=bnb_config)

tokenizer.add_special_tokens({"additional_special_tokens": ['bicol']}, replace_additional_special_tokens=False)

#tokenizer.pad_token = tokenizer.eos_token

1

In [20]:
lora_config = LoraConfig(
    r=8,
    target_modules=["q_proj", "v_proj", "k_proj"],
    lora_alpha=16,
    lora_dropout=0.1,
    bias="none",
    task_type="SEQ_2_SEQ_LM",
)

peft_model = get_peft_model(model, lora_config)
peft_model.print_trainable_parameters()

trainable params: 1,769,472 || all params: 616,843,264 || trainable%: 0.2869


In [21]:
tokens = tokenizer("How is you day today?", return_tensors="pt").to(model.device)
translated_tokens = model.generate(
    **tokens, forced_bos_token_id=tokenizer.convert_tokens_to_ids("tgl_Latn"), max_length=30,
)
text = tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)

In [22]:
text

['Kumusta ka sa araw ngayon?']

In [23]:
import pandas as pd
import datasets

parallel_corpora = pd.read_csv("english-to-bicol-corpora.csv")

def preprocess(batch):
    model_inputs = tokenizer(
        batch["language1_text"],
        truncation=True,
        padding="max_length",
        max_length=128,
        
    )

    labels = tokenizer(
        batch["language2_text"],
        truncation=True,
        padding="max_length",
        max_length=128,
    )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

corpora = parallel_corpora.sample(frac=1, random_state=42)
train_df = corpora.sample(frac=0.99, random_state=42)
eval_df = corpora.drop(train_df.index).reset_index(drop=True)
train_df = train_df.reset_index(drop=True)

train_dataset = datasets.Dataset.from_pandas(train_df)
eval_dataset = datasets.Dataset.from_pandas(eval_df)

train_dataset_processed = train_dataset.map(preprocess, batched=True, remove_columns=['language1_text', 'language2_text'])
eval_dataset_processed = eval_dataset.map(preprocess, batched=True, remove_columns=['language1_text', 'language2_text'])

Map:   0%|          | 0/28100 [00:00<?, ? examples/s]

Map:   0%|          | 0/284 [00:00<?, ? examples/s]

In [24]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="trained-nllb-en-to-bicol",
    learning_rate=5e-4,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    eval_strategy="steps",
    eval_steps=200,
    save_strategy="steps",
    save_steps=200,
    load_best_model_at_end=True,
    logging_strategy="steps",
    logging_steps=1,
    logging_first_step=True,
    report_to="none",
    predict_with_generate=True,
)

In [25]:
from transformers import Seq2SeqTrainer
from peft import LoraConfig
import torch

import numpy as np

import evaluate

metric = evaluate.load("accuracy")
bleu = evaluate.load("bleu")

gl_eval_pred = None

#def compute_metrics(eval_pred):
#    logits, labels = eval_pred
#    # If logits are None (some eval configurations), return empty dict
#    if logits is None:
#        return {}
#    # Convert logits to predicted token ids. Logits may be (batch, seq, vocab)
#    preds = np.argmax(logits, axis=-1)
#    labels = np.array(labels)
#    labels_str = tokenizer.batch_decode(labels, skip_special_tokens=True)
#    preds = np.array(preds)
#    # Mask out label padding (we use -100 for padding labels). Only keep positions where label != -100
#    mask = labels != -100
#    if mask.sum() == 0:
#        return {}
#    preds_flat = preds[mask]
#    labels_flat = labels[mask]
#
#    pred_str = tokenizer.batch_decode(preds, skip_special_tokens=True)
#    bleu = bleu.compute(predictions=pred_str, references=[[l] for l in labels_str])
#    return {
#        "accuracy": metric.compute(predictions=preds_flat.astype(np.int32), references=labels_flat.astype(np.int32)),
#        "bleu": bleu["bleu"],
#    }

gl_decoded_preds = None
gl_decoded_labels = None
gl_eval_preds = None

def compute_metrics(eval_preds):
    global gl_eval_preds
    gl_eval_preds = eval_preds
    preds, labels = eval_preds

    labels = np.where(labels == -100, tokenizer.pad_token_id, labels)

    if isinstance(preds, torch.Tensor):
        preds = preds.tolist()
    if isinstance(labels, torch.Tensor):
        labels = labels.tolist()
    # Convert token IDs to text
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    global gl_decoded_preds, gl_decoded_labels
    gl_decoded_preds = decoded_preds
    gl_decoded_labels = decoded_labels

    # sacrebleu expects list of predictions, list of list of references
    result = bleu.compute(predictions=decoded_preds,
                            references=[[l] for l in decoded_labels])
    gl_eval_preds = result
    return {"bleu": result["bleu"]}

trainer = Seq2SeqTrainer(
    model=peft_model,
    args=training_args,
    train_dataset=train_dataset_processed,
    eval_dataset=eval_dataset_processed,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

C:\Users\rek\AppData\Local\Temp\ipykernel_26412\4143228653.py:67: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [27]:
trainer.train(resume_from_checkpoint=True)

RuntimeError: Error(s) in loading state_dict for PeftModelForSeq2SeqLM:
	size mismatch for base_model.model.model.encoder.layers.0.self_attn.k_proj.lora_A.default.weight: copying a param with shape torch.Size([4, 1024]) from checkpoint, the shape in current model is torch.Size([8, 1024]).
	size mismatch for base_model.model.model.encoder.layers.0.self_attn.k_proj.lora_B.default.weight: copying a param with shape torch.Size([1024, 4]) from checkpoint, the shape in current model is torch.Size([1024, 8]).
	size mismatch for base_model.model.model.encoder.layers.0.self_attn.v_proj.lora_A.default.weight: copying a param with shape torch.Size([4, 1024]) from checkpoint, the shape in current model is torch.Size([8, 1024]).
	size mismatch for base_model.model.model.encoder.layers.0.self_attn.v_proj.lora_B.default.weight: copying a param with shape torch.Size([1024, 4]) from checkpoint, the shape in current model is torch.Size([1024, 8]).
	size mismatch for base_model.model.model.encoder.layers

In [ ]:
trainer.save_model("nllb-en-to-bicol-seq2seq-model")    

In [ ]:
generated_tokens = trainer.model.generate(**tokens)